# Sample FastAPI Application
This notebook demonstrates a simple FastAPI REST API that can be deployed to Cloud Run.
It includes endpoints for a simple todo list application.

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import uvicorn

In [ ]:
# Define data models
class TodoItem(BaseModel):
    id: Optional[int] = None
    title: str
    description: Optional[str] = None
    completed: bool = False

In [ ]:
# Initialize FastAPI app
app = FastAPI(
    title="Todo API",
    description="A simple Todo API deployed from Jupyter Notebook",
    version="1.0.0"
)

# In-memory storage
todos: List[TodoItem] = []
next_id = 1

In [ ]:
@app.get("/")
def read_root():
    """Root endpoint"""
    return {
        "message": "Welcome to Todo API",
        "endpoints": [
            "/todos - GET all todos",
            "/todos/{id} - GET a specific todo",
            "/todos - POST create a new todo",
            "/todos/{id} - PUT update a todo",
            "/todos/{id} - DELETE a todo"
        ]
    }

In [ ]:
@app.get("/health")
def health_check():
    """Health check endpoint"""
    return {"status": "healthy", "total_todos": len(todos)}

In [ ]:
@app.get("/todos", response_model=List[TodoItem])
def get_todos(completed: Optional[bool] = None):
    """Get all todos, optionally filtered by completion status"""
    if completed is None:
        return todos
    return [todo for todo in todos if todo.completed == completed]

In [ ]:
@app.get("/todos/{todo_id}", response_model=TodoItem)
def get_todo(todo_id: int):
    """Get a specific todo by ID"""
    for todo in todos:
        if todo.id == todo_id:
            return todo
    raise HTTPException(status_code=404, detail="Todo not found")

In [ ]:
@app.post("/todos", response_model=TodoItem, status_code=201)
def create_todo(todo: TodoItem):
    """Create a new todo"""
    global next_id
    todo.id = next_id
    next_id += 1
    todos.append(todo)
    return todo

In [ ]:
@app.put("/todos/{todo_id}", response_model=TodoItem)
def update_todo(todo_id: int, updated_todo: TodoItem):
    """Update a todo"""
    for i, todo in enumerate(todos):
        if todo.id == todo_id:
            updated_todo.id = todo_id
            todos[i] = updated_todo
            return updated_todo
    raise HTTPException(status_code=404, detail="Todo not found")

In [ ]:
@app.delete("/todos/{todo_id}")
def delete_todo(todo_id: int):
    """Delete a todo"""
    global todos
    original_length = len(todos)
    todos = [todo for todo in todos if todo.id != todo_id]
    if len(todos) == original_length:
        raise HTTPException(status_code=404, detail="Todo not found")
    return {"message": "Todo deleted successfully"}

In [ ]:
# Run the server (for local testing)
if __name__ == "__main__":
    import os
    port = int(os.getenv("PORT", 8080))
    uvicorn.run(app, host="0.0.0.0", port=port)